### Imports

In [41]:
import lightgbm as lgb
import numpy as np
import pandas as pd
from pprint import pprint
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
import seaborn as sns
from tensorflow import keras
import tensorflow as tf
import warnings
from tqdm import tqdm, tqdm_notebook
from sklearn.model_selection import cross_val_score

from sklearn.feature_selection import RFE
from sklearn.feature_selection import SelectPercentile
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
plt.rc('font', family=fm.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()) # for Windows OS user
import datetime
import os

In [42]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor
from catboost import CatBoostRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import BayesianRidge
from tensorflow import keras
from scipy.stats.mstats import gmean
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
import sys, warnings
if not sys.warnoptions: warnings.simplefilter("ignore")

In [43]:
import datetime
from sklearn.model_selection import StratifiedKFold, KFold
import math

In [44]:
from sklearn.decomposition import PCA

In [45]:
from sklearn.preprocessing import PowerTransformer, StandardScaler, MinMaxScaler, RobustScaler, QuantileTransformer

### Read Data

In [46]:
df_y_train_c = pd.read_csv('y_train.csv')

In [47]:
df_x_train = pd.read_csv('X_train.csv', encoding='cp949')
df_x_test = pd.read_csv('X_test.csv', encoding='cp949')
df_y_train = pd.read_csv('y_train.csv')
df_y_train['age'] = df_y_train['group'].str[1:].astype(int)
IDtrain = pd.DataFrame({'custid': df_x_train.custid.unique()})
IDtest = pd.DataFrame({'custid': df_x_test.custid.unique()})

In [98]:
df_x_test

,custid,sales_month,sales_day,sales_dayofweek,sales_time,str_nm,goodcd,brd_nm,corner_nm,pc_nm,part_nm,team_nm,buyer_nm,import_flg,tot_amt,dis_amt,net_amt,inst_mon,inst_fee
0,30001,5,6,토,1703,본점,4220242014400,ELLE,내의,내의란제리,"케주얼,구두,아동",의류패션팀,니트단품,0,-64700,-3240,-61460,1,0
1,30001,5,6,토,1703,본점,4220242014400,ELLE,내의,내의란제리,"케주얼,구두,아동",의류패션팀,니트단품,0,63600,3180,60420,1,0
2,30001,5,21,일,1653,본점,4116900003200,사보,캐쥬얼구두,남여구두,"케주얼,구두,아동",의류패션팀,피혁A,0,150000,7500,142500,3,0
3,30001,5,21,일,1700,본점,4109350003200,탠디,캐쥬얼구두,남여구두,"케주얼,구두,아동",의류패션팀,피혁A,0,135000,6750,128250,3,0
4,30001,5,31,수,1710,본점,4116900003200,사보,캐쥬얼구두,남여구두,"케주얼,구두,아동",의류패션팀,피혁A,0,138000,6900,131100,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
414950,49993,13,31,수,1750,신촌점,4405551020474,톰키드,아동,아동,아동문화,잡화가용팀,유아동복,0,20000,0,20000,1,0
414951,49993,13,31,수,1833,신촌점,2139140008300,폴로화장품,향수,화장품,패션잡화,잡화가용팀,화장품,0,70000,3500,66500,1,0
414952,49994,16,14,토,1750,본점,4230120011274,스테파넬,영트랜드,영트렌디,"케주얼,구두,아동",의류패션팀,영캐주얼,0,39000,0,39000,1,0
414953,49994,16,14,토,1810,본점,4409271026010,써스데이아일앤드,진케주얼,진케주얼,"케주얼,구두,아동",의류패션팀,유니캐주얼,0,34200,0,34200,1,0


# log 데이터 전처리

In [48]:
tr = pd.concat([df_x_train, df_x_test])
tr['real_amt']= tr.tot_amt / tr.inst_mon
tr['dist_rate'] = (tr['dis_amt']/tr['tot_amt'])*100

In [49]:
id_data = pd.DataFrame({'custid': tr.custid.unique()})

In [50]:
tr['sales_hour'] = tr['sales_time']//100
tr['sales_min'] = tr['sales_time']%100

In [51]:
tr["sales_date"] = np.where(tr["sales_month"] >9 , 
                            (tr["sales_month"].astype(str)),("0"+tr["sales_month"].astype(str)))


In [52]:
tr["sales_date"] = np.where(tr["sales_day"]>9,(tr["sales_date"]+tr["sales_day"].astype(str)),
                            (tr["sales_date"]+"0"+tr["sales_day"].astype(str))
                            )

In [53]:
tr["sales_month"] = np.where(tr["sales_month"]>12 , tr["sales_month"]-12, tr["sales_month"])

In [54]:
tr["time"] = pd.to_datetime(tr.sales_time, format = "%H%M")

In [55]:
tr["refund"] = np.where(tr["net_amt"]<0,1,0)

In [56]:
tr['sales_month_ver1'] = tr['sales_month']
tr['sales_month'] = tr['sales_month'].apply(lambda x : x-12 if x > 12 else x )
tr['sales_hour'] = tr['sales_time'].apply(lambda x : x//100 )
tr['sales_sec'] = tr['sales_time'].apply(lambda x : x%100 )
tr['total_sec'] = tr['sales_time'].apply(lambda x : x//100*60 + x%100 )
tr['환불여부'] = tr['tot_amt'].apply(lambda x : 1 if x < 0 else 0 )
tr['refund'] = tr['tot_amt'].apply(lambda x : abs(x) if x < 0 else 0 )
tr['tot_amt'] = tr['tot_amt'] .apply(lambda x : 0 if x < 0 else x )
tr['real_amt'] = ( tr['tot_amt'] / tr['inst_mon'] ).apply(lambda x : math.trunc(x)) 
tr['sales_date'] =tr['sales_month_ver1'].astype(str).apply(lambda x : "0"+x if len(x) == 1  else x ) +tr['sales_day'].astype(str).apply(lambda x : "0"+x if len(x) == 1 else  x )
tr['sales_date'] = tr['sales_date'].astype(int)

In [57]:
df = pd.merge(tr, df_y_train_c, on = 'custid')
df['age'] = df['group'].str[1:].astype(int)
tr['age_group'] = df['age'].apply(lambda x : 'twenty' if (x>=20) & (x<30)
                                     else 'thirty' if (x>=30) & (x<40)
                                     else 'forty' if (x>=40) & (x<50)
                                     else 'fifty' )

In [58]:
twenty_prefer_brd = tr[tr['age_group'] == 'twenty'].brd_nm.value_counts().index[1:].to_list()
thirty_prefer_brd = tr[tr['age_group'] == 'thirty'].brd_nm.value_counts().index[1:].to_list()
forty_prefer_brd = tr[tr['age_group'] == 'forty'].brd_nm.value_counts().index[1:].to_list()
fifty_prefer_brd = tr[tr['age_group'] == 'fifty'].brd_nm.value_counts().index[1:].to_list()

def prefer_brd(x, list):
    for i in range(len(list)):
        if(x == list[i]):
            return len(list)-i

tr['20_weight'] = tr['brd_nm'].apply(lambda x: prefer_brd(x, twenty_prefer_brd)).fillna(0)
tr['30_weight'] = tr['brd_nm'].apply(lambda x: prefer_brd(x, thirty_prefer_brd)).fillna(0)
tr['40_weight'] = tr['brd_nm'].apply(lambda x: prefer_brd(x, forty_prefer_brd)).fillna(0)
tr['50_weight'] = tr['brd_nm'].apply(lambda x: prefer_brd(x, fifty_prefer_brd)).fillna(0)


In [59]:
tr["age_weight_sum"] = tr['20_weight']+tr['30_weight']+tr['40_weight']+tr['50_weight']

In [60]:
tr["sales_season"] = tr['sales_month'].apply(lambda x : 'SPRING' if (x>=3) and (x<=5)
                                                  else 'SUMMER' if (x>=6) and (x<=8)
                                                  else 'FALL' if (x>=9) and (x<=11)
                                                  else 'WINTER')

In [61]:
tr['tot_amt>=0'] = tr['tot_amt'].apply(lambda x : x if x>=0 else 0)
tr['dis_amt>=0'] = tr['dis_amt'].apply(lambda x : x if x>=0 else 0)
tr['net_amt>=0'] = tr['net_amt'].apply(lambda x : x if x>=0 else 0)

# features 생성

In [62]:
features = []

In [63]:
from gensim.models import word2vec
import random

In [64]:
def oversample(x, n):
    lst = []
    random.seed(100)
    for i in x:
        tmp = []
        for j in range(n):
            random.shuffle(i)
            tmp += list(i)
            lst.append(tmp)
    return lst

# W2V goodcd

In [65]:
df=tr.reset_index().drop('index',axis=1).copy().reset_index()
df['goodcd']=df['goodcd'].astype('str')
train_data=list(df.groupby('custid')['goodcd'].unique())

In [66]:
w2v_input = oversample(train_data, 5)

In [67]:
import gensim

In [68]:
w2v = word2vec.Word2Vec(sentences = w2v_input, vector_size = 100, window = 5,workers=6, min_count = 1, sg = 1)

In [69]:
train_mean_vector = []
for words in tqdm(train_data):
    tmp = np.zeros(100)
    cnt = 0
    for word in words:
        tmp += w2v.wv[word]
        cnt += 1
    tmp /= cnt
    train_mean_vector.append(tmp)
train_mean_vector = np.array(train_mean_vector)

100%|█████████████████████████████████████████████████████████████████████████| 35967/35967 [00:02<00:00, 14379.66it/s]


In [70]:
w2v_good= pd.DataFrame(train_mean_vector)

# W2V brd_nm

In [71]:
df=tr.reset_index().drop('index',axis=1).copy().reset_index()
train_data=list(df.groupby('custid')['brd_nm'].unique())

In [72]:
w2v_input = oversample(train_data, 5)
w2v = word2vec.Word2Vec(sentences = w2v_input, vector_size = 100, window = 5,workers=6, min_count = 1, sg = 1)

In [73]:
train_mean_vector = []
for words in tqdm(train_data):
    tmp = np.zeros(100)
    cnt = 0
    for word in words:
        tmp += w2v.wv[word]
        cnt += 1
    tmp /= cnt
    train_mean_vector.append(tmp)
train_mean_vector = np.array(train_mean_vector)

100%|█████████████████████████████████████████████████████████████████████████| 35967/35967 [00:02<00:00, 15509.50it/s]


In [74]:
w2v_brd = pd.DataFrame(train_mean_vector)

# W2V corner_nm

In [75]:
df=tr.reset_index().drop('index',axis=1).copy().reset_index()
train_data=list(df.groupby('custid')['corner_nm'].unique())

In [76]:
w2v_input = oversample(train_data, 5)
w2v = word2vec.Word2Vec(sentences = w2v_input, vector_size = 100, window = 5,workers=6, min_count = 1, sg = 1)

In [77]:
train_mean_vector = []
for words in tqdm(train_data):
    tmp = np.zeros(100)
    cnt = 0
    for word in words:
        tmp += w2v.wv[word]
        cnt += 1
    tmp /= cnt
    train_mean_vector.append(tmp)
train_mean_vector = np.array(train_mean_vector)

100%|█████████████████████████████████████████████████████████████████████████| 35967/35967 [00:01<00:00, 21174.06it/s]


In [78]:
w2v_coner = pd.DataFrame(train_mean_vector)

- 환불데이터

In [79]:
t = tr.groupby("custid")["refund"].agg([("refund_count","sum")]).reset_index()
t["refund_bool"] = np.where(t["refund_count"]>0,1,0)
features.append(t)

- cn  구매하는 나이

In [80]:
data = pd.concat([id_data,df_y_train],axis = 1)
data = data.iloc[:len(df_y_train)]

In [89]:
data = data.iloc[:,[0,2,3]]

In [100]:
data

,custid,group,age
0,0,M30,30.0
1,2,M30,30.0
2,3,F20,20.0
3,4,F20,20.0
4,5,M40,40.0
...,...,...,...
21582,29995,M50,50.0
21583,29996,F20,20.0
21584,29997,M30,30.0
21585,29998,F20,20.0


In [91]:
tr_data = pd.merge(tr,data,on="custid")

In [103]:
tr_cor_mean = tr_data.groupby("corner_nm")["age"].mean()

In [104]:
tr_cor_mean # 주구매 코너를 이걸로 매핑해주자

corner_nm
DC캐주얼      32.911392
GBR  지원    41.000000
L/B침구      31.753555
N/B침구      32.178030
NB제화       31.276596
             ...    
홈데코        30.836820
홈쇼핑        50.000000
화장잡화       33.248731
화장품        27.070234
훼미닌부틱      42.785146
Name: age, Length: 308, dtype: float64

- brd 구매 평균나이

In [105]:
tr_brd_mean = tr_data.groupby("brd_nm")["age"].mean()

- pc 구매 평균나이

In [106]:
tr_pc_mean = tr_data.groupby("pc_nm")["age"].mean()

- 주구매 요일 나이평균

In [107]:
tr_day_mean = tr_data.groupby("sales_dayofweek")["age"].mean()

In [108]:
def refund_check(x):
    if x<0:
        return x
    else:
        return 0

- 환불 총액, 평균 , 최대

In [109]:
copy_df = tr.copy()
copy_df["tot_amt_refund"] = copy_df.net_amt.apply(refund_check)
t = copy_df.groupby("custid")["tot_amt_refund"].agg([("amt_refund","sum"),
                                                    ("amt_refund_mean","mean"),
                                                    ("amt_refund_max","min")]).reset_index()
features.append(t)

- 최애브랜드 사용수

In [110]:
def g(x):
    y = x.value_counts()
    y = y.iloc[0]
    return y

f = tr.groupby('custid')['brd_nm'].agg([('love_brd_count', g)]).reset_index()
f = f.fillna(0)
features.append(f)

- 연령대별 브랜드 선호도에 따른 가중치 비율

In [111]:
f = tr.groupby('custid')['20_weight', '30_weight', '40_weight', '50_weight', 'age_weight_sum'].sum()

f['20_weight_ratio'] = f['20_weight'] / f['age_weight_sum']
f['30_weight_ratio'] = f['30_weight'] / f['age_weight_sum']
f['40_weight_ratio'] = f['40_weight'] / f['age_weight_sum']
f['50_weight_ratio'] = f['50_weight'] / f['age_weight_sum']


f = f.fillna(0)
features.append(f)

- 시간대별 방문횟수

In [112]:
t = pd.pivot_table(tr, index='custid', columns='sales_hour', values='tot_amt', 
                   aggfunc=np.size).fillna(0).astype(int).reset_index()
features.append(t)

- 시간대별 구매금액

In [113]:
t = pd.pivot_table(tr, index='custid', columns='sales_hour', values='tot_amt', 
                   aggfunc="sum").fillna(0).astype(int).reset_index().drop([0,1,8,9],axis = 1)
features.append(t)

- 날짜별 방문횟수

In [114]:
t = pd.pivot_table(tr, index='custid', columns='sales_day', values='tot_amt', 
                   aggfunc=np.size).fillna(0).astype(int).reset_index()

In [115]:
f = id_data.copy()
f["month_start"] = 0
f["month_mid"] = 0
f["month_end"] = 0 
for i in [x for x in range(1,11)]:
    f["month_start"] += t[i]
    f["month_mid"] += t[i+10]
    f["month_end"] += t[i+20]
f["month_end"] += t[31]
features.append(f)

- 날짜별 구매금액

In [116]:
t = pd.pivot_table(tr, index='custid', columns='sales_day', values='tot_amt', 
                   aggfunc="sum").fillna(0).astype(int).reset_index()

In [117]:
f = id_data.copy()
f["month_start"] = 0
f["month_mid"] = 0
f["month_end"] = 0 
for i in [x for x in range(1,11)]:
    f["month_start"] += t[i]
    f["month_mid"] += t[i+10]
    f["month_end"] += t[i+20]
f["month_end"] += t[31]
features.append(f)

- 월간별 날짜횟수

In [118]:
t = pd.pivot_table(tr, index='custid', columns='sales_month', values='tot_amt', 
                   aggfunc=np.size).fillna(0).astype(int).reset_index()
features.append(t)

- 월간 구매금액

In [119]:
t = pd.pivot_table(tr, index='custid', columns='sales_month', values='tot_amt', 
                   aggfunc="sum").fillna(0).astype(int).reset_index()
features.append(t)

In [120]:
#쇼핑시간
time_sum = tr.groupby(['sales_date','custid'])['time'].agg([('time', ['min','max'])]).reset_index()
time_sum['shopping_time'] = (time_sum['time']['max'] - time_sum['time']['min']).dt.total_seconds()
time_sum.drop(['sales_date','time'], axis=1, inplace=True,level=0)
time_sum = time_sum.groupby(['custid'])['shopping_time'].agg([('shopping_time_mean','mean')]).reset_index()
features.append(time_sum)

In [121]:
#평균할인율
f = tr.groupby('custid')['dist_rate'].agg([('dis_rate', 'mean')]).reset_index()
features.append(f)

In [122]:
#평균시간
f = tr.groupby(['custid'])['sales_time'].agg([('sales_time', 'mean')]).reset_index()
features.append(f)

In [123]:
#남성파트
df = tr.groupby(['custid','part_nm'])['tot_amt'].agg([('tot_amt_part', 'sum')]).reset_index()
df['part_nm'] = np.where(df.part_nm.str.contains('남성'), '남성', '비남성')
df = df.pivot_table(values='tot_amt_part', index=df.custid, columns='part_nm', aggfunc='first',fill_value=0).reset_index()
df['남성part'] = (df['남성'] / (df['남성'] + df['비남성'])) * 100
df = df.fillna(0)
features.append(df)

In [124]:
#화장품구매비율
df = tr.groupby(['custid','corner_nm'])['tot_amt'].agg([('tot_amt_corner', 'sum')]).reset_index()
df['corner_nm'] = np.where(df.corner_nm.str.contains('화장품'), '화장품', '비화장품')
df = df.pivot_table(values='tot_amt_corner', index=df.custid, columns='corner_nm', aggfunc='first',fill_value=0).reset_index()
df['화장품비율'] = (df['화장품'] / (df['화장품'] + df['비화장품'])) * 100
df = df.fillna(0)
features.append(df)

In [125]:
#할부대비평균실구매
f = tr.groupby('custid')['real_amt'].agg([('real_amt', 'mean')]).reset_index()
features.append(f)

In [126]:
#평균구매상품종류
df =tr.groupby(['custid','goodcd'])['tot_amt'].agg([('good_count', 'count')]).reset_index()
f = df.groupby(['custid'])['good_count'].agg([('good_count_mean', 'mean')]).reset_index()
features.append(f)

In [127]:
#지역
df = tr.groupby(['custid','str_nm'])['tot_amt'].agg([('tot_amt_str', 'sum')]).reset_index()
df =df.pivot_table(values='tot_amt_str', index=df.custid, columns='str_nm', aggfunc='first',fill_value=0).reset_index()
features.append(df)

In [128]:
#팀별
df = tr.groupby(['custid','team_nm'])['tot_amt'].agg([('tot_amt_team', 'sum')]).reset_index()
df =df.pivot_table(values='tot_amt_team', index=df.custid, columns='team_nm', aggfunc='first',fill_value=0).reset_index()
features.append(df)

In [129]:
#총구매수입상품
df = tr.groupby(['custid'])['import_flg'].agg([('import_flg_sum', 'sum')]).reset_index()
features.append(df)

In [130]:
#파트
df = tr.groupby(['custid','part_nm'])['tot_amt'].agg([('tot_amt_part', 'sum')]).reset_index()
df =df.pivot_table(values='tot_amt_part', index=df.custid, columns='part_nm', aggfunc='first',fill_value=0).reset_index()
features.append(df)

In [131]:
#코너
df = tr.groupby(['custid','corner_nm'])['tot_amt'].agg([('tot_amt_corner', 'sum')]).reset_index()
df =df.pivot_table(values='tot_amt_corner', index=df.custid, columns='corner_nm', aggfunc='first',fill_value=0).reset_index()
features.append(df)

In [132]:
#pc
df = tr.groupby(['custid','pc_nm'])['tot_amt'].agg([('tot_amt_pc', 'sum')]).reset_index()
df =df.pivot_table(values='tot_amt_pc', index=df.custid, columns='pc_nm', aggfunc='first',fill_value=0).reset_index()
features.append(df)

In [133]:
#총구매액
f = tr.groupby('custid')['tot_amt'].agg([('총구매액', 'sum')]).reset_index()
features.append(f)

In [134]:
#구매건수
f = tr.groupby('custid')['tot_amt'].agg([('구매건수', 'size')]).reset_index()
features.append(f)

In [135]:
#평균구매가격

f = tr.groupby('custid')['tot_amt'].agg([('평균구매가격', "mean")]).reset_index()
features.append(f)

In [136]:
#평균할부개월수
f = tr.groupby('custid')['inst_mon'].agg([('평균할부개월수', 'mean')]).reset_index()
f.iloc[:,1] = f.iloc[:,1].apply(round, args=(1,))
features.append(f)

In [137]:
#구매상품다양성
n = tr.corner_nm.nunique()
f = tr.groupby('custid')['brd_nm'].agg([('구매상품다양성', lambda x: len(x.unique()) / n)]).reset_index()
features.append(f)

In [138]:
#수입상품_구매비율
x = tr[tr['import_flg'] == 1].groupby('custid').size() / tr.groupby('custid').size()
f = x.reset_index().rename(columns={0: '수입상품_구매비율'}).fillna(0)
f.iloc[:,1] = (f.iloc[:,1]*100).apply(round, args=(1,))
features.append(f)

In [139]:
#일평균구매액
test2 = tr.groupby(['sales_date','custid'])['tot_amt'].agg([('day_amt', 'sum')]).reset_index()
test2 = test2.groupby(['custid'])['day_amt'].agg([('일평균구매액', 'mean')]).reset_index()
features.append(test2)

In [140]:
#내점일수 

f = tr.groupby(by = 'custid')['sales_time'].agg([('내점일수','nunique')]).reset_index()
features.append(f)

In [141]:
#일평균구매건
df = tr.groupby(['sales_date','custid'])['custid'].agg([('day_visit', 'count')]).reset_index()
f = df.groupby(['custid'])['day_visit'].agg([('일평균구매건', 'mean')]).reset_index()
features.append(f)

In [142]:
#아동상품 구매건수
f = tr[tr.tot_amt > 0].groupby('custid')['part_nm'].agg([('baby_sales', lambda x: list(x).count('아동')+list(x).count('케주얼,구두,아동')+list(x).count('아동문화')+list(x).count('아동,스포츠'))]).reset_index()
features.append(f)

In [143]:
#상위 브랜드 40개의 구매빈도
top_brd = tr["brd_nm"].value_counts()[:40].index
df = tr.groupby(['custid','brd_nm'])["custid"].agg([('brd_count', 'count')]).reset_index()
df =df.pivot_table(values='brd_count', index=df.custid, columns="brd_nm", aggfunc='count',fill_value=0).reset_index()
df = pd.concat([df["custid"] ,df[top_brd]],axis= 1)
features.append(df)

In [144]:
#상위 브랜드 40개의 구매금액
df = tr.groupby(['custid','brd_nm'])["tot_amt"].agg([('tot_amt_brd', 'sum')]).reset_index()
df =df.pivot_table(values='tot_amt_brd', index=df.custid, columns="brd_nm", aggfunc='sum',fill_value=0).reset_index()
df = pd.concat([df["custid"] ,df[top_brd]],axis= 1)
features.append(df)

In [145]:
#주말방문비율
day = {'월' : 0 , '화' : 1 , '수' : 2 , '목' : 3 , '금' : 4 , '토': 5 , '일' : 6 }

tr['sales_dayofweek_num'] = tr['sales_dayofweek'].apply(lambda x : day[x] )

f = tr.groupby('custid')['sales_dayofweek_num'].agg([
    ('주말방문비율', lambda x: np.mean(x >4))]).reset_index()

features.append(f)

In [146]:
#계절별 구매 건수
def season(k):
    if 3 <= k <= 5 :
        return('봄')
    elif 6 <= k <= 8 :
        return('여름')
    elif 9 <= k <= 11 :    
        return('가을')
    else :
        return('겨울')
df = tr.copy()
df["season"] = df.sales_month.apply(season)
f = pd.pivot_table(df, index='custid', columns='season', values='tot_amt', 
                   aggfunc=np.size, fill_value=0).reset_index()
features.append(f)

In [147]:
#계절별 구매 금액
df = tr.copy()
df["season"] = df.sales_month.apply(season)
f = pd.pivot_table(df, index='custid', columns='season', values='tot_amt', 
                   aggfunc="sum", fill_value=0).reset_index()
features.append(f)

In [148]:
#아침점심저녁별 구매건수
def f2(x):
    if 9 <= x <= 12 :
        return('아침')
    elif 13 <= x <= 16 :
        return('점심')
    else :
        return('저녁')  # datatime 필드가 시간 형식에 맞지 않은 값을 갖는 경우 저녁시간으로 처리
df = tr.copy()
df["goodmea"] = df.sales_hour.apply(f2)
f = pd.pivot_table(df, index='custid', columns='goodmea', values='tot_amt', 
                   aggfunc=np.size).reset_index()
features.append(f)

In [149]:
#아침점심저녁별 구매금액
df = tr.copy()
df["goodmea"] = df.sales_hour.apply(f2)
f = pd.pivot_table(df, index='custid', columns='goodmea', values='tot_amt', 
                   aggfunc="sum").reset_index()
features.append(f)

In [150]:
#주방문요일
f = tr.groupby('custid')['sales_dayofweek'].agg([('주방문요일', lambda x: x.value_counts().index[0])]).reset_index()
f = pd.get_dummies(f, columns=['주방문요일'])

features.append(f)

In [151]:
#평균 쇼핑시간
f = tr.groupby('custid')['total_sec'].agg([
    ('평균쇼핑시간', lambda x: (x.max() - x.min()) / x.nunique())]).reset_index()

features.append(f)

In [152]:
#일별 구매건수
f = pd.pivot_table(tr, index='custid', columns='sales_day', values='tot_amt', 
                   aggfunc=np.size).fillna(0).astype(int).reset_index()
features.append(f)

In [153]:
#일별 구매금액
f = pd.pivot_table(tr, index='custid', columns='sales_day', values='tot_amt', 
                   aggfunc="sum").fillna(0).astype(int).reset_index()
features.append(f)

In [154]:
#월별 구매건수
f = pd.pivot_table(tr, index='custid', columns='sales_month', values='tot_amt', 
                   aggfunc=np.size).fillna(0).astype(int).reset_index()
features.append(f)

In [155]:
#월별 구매금액
f = pd.pivot_table(tr, index='custid', columns='sales_month', values='tot_amt', 
                   aggfunc="sum").fillna(0).astype(int).reset_index()
features.append(f)

In [156]:
data = pd.concat([id_data,df_y_train],axis = 1)
data = data.iloc[:len(df_y_train)]

In [160]:
data = data.iloc[:,[0,2,3]]

In [161]:
data

,custid,group,age
0,0,M30,30.0
1,2,M30,30.0
2,3,F20,20.0
3,4,F20,20.0
4,5,M40,40.0
...,...,...,...
21582,29995,M50,50.0
21583,29996,F20,20.0
21584,29997,M30,30.0
21585,29998,F20,20.0


In [162]:
tr_data = pd.merge(tr,data,on="custid")

In [163]:
tr_cor_mean = dict(tr_data.groupby("corner_nm")["age"].mean())
tr_brd_mean = dict(tr_data.groupby("brd_nm")["age"].mean())
tr_pc_mean = dict(tr_data.groupby("pc_nm")["age"].mean())
tr_day_mean = dict(tr_data.groupby("sales_dayofweek")["age"].mean())

In [164]:
brd_data = tr.groupby("custid")["pc_nm"].agg([("pc","first")])

In [165]:
f = brd_data["pc"].apply(lambda x: tr_pc_mean[x])
features.append(f)

In [166]:
d = tr.groupby("custid")["corner_nm"].agg([("cor","first")])
f = d["cor"].apply(lambda x: tr_cor_mean[x])
features.append(f)

In [167]:
d = tr.groupby("custid")["sales_dayofweek"].agg([("day","first")])
f = d["day"].apply(lambda x: tr_day_mean[x])
features.append(f)

In [168]:
X_train = pd.DataFrame({'custid': df_x_train.custid.unique()})
for f in features :
    X_train = pd.merge(X_train, f, how='left',on='custid')
display(X_train.shape)

X_test = pd.DataFrame({'custid': df_x_test.custid.unique()})
for f in features :
    X_test = pd.merge(X_test, f, how='left',on='custid')

(21587, 713)

In [169]:
X_train['평균내점구매액'] = X_train['총구매액']/X_train['내점일수']
X_train['주중방문비율'] = (100 - X_train["주말방문비율"])
X_train['국내상품_구매비율'] = (100 - X_train['수입상품_구매비율'])
X_train['할부구매가격'] = X_train['평균구매가격'] / X_train['평균할부개월수']
X_train['구매상품다양성'] = X_train['총구매액'] / X_train['구매상품다양성']
X_train['주말방문수'] = (X_train['주말방문비율'] * X_train['내점일수']) / 100
X_train['주말방문수'] = X_train['주말방문수'].astype('int64')
X_train['주중방문수'] = X_train['내점일수'] - X_train['주말방문수']
X_train['주중방문수'] = X_train['주중방문수'].astype('int64')
X_train['내점당편균구매건수'] = X_train['구매건수']/X_train['내점일수']
X_train['주중구매액'] = X_train['총구매액']*(X_train['주중방문비율']/100)
X_train['주말구매액'] = X_train['총구매액'] - X_train['주중구매액']

In [170]:
X_test['평균내점구매액'] = X_test['총구매액']/X_test['내점일수']
X_test['주중방문비율'] = (100 - X_test['주말방문비율'])
X_test['국내상품_구매비율'] = (100 - X_test['수입상품_구매비율'])
X_test['할부구매가격'] = X_test['평균구매가격'] / X_test['평균할부개월수']
X_test['구매상품다양성'] = X_test['총구매액'] / X_test['구매상품다양성']
X_test['주말방문수'] = (X_test['주말방문비율'] * X_test['내점일수']) / 100
X_test['주말방문수'] = X_test['주말방문수'].astype('int64')
X_test['주중방문수'] = X_test['내점일수'] - X_test['주말방문수']
X_test['주중방문수'] = X_test['주중방문수'].astype('int64')
X_test['내점당편균구매건수'] = X_test['구매건수']/X_test['내점일수']
X_test['주중구매액'] = X_test['총구매액']*(X_test['주중방문비율']/100)
X_test['주말구매액'] = X_test['총구매액'] - X_test['주중구매액']

# kmean 클러스터링

In [171]:
features__ = X_train.drop(columns='custid',axis=1).fillna(0)
features_t__= X_test.drop(columns='custid',axis=1).fillna(0)

In [174]:
from sklearn.cluster import KMeans

class KMeansFeaturizer:
    """ 숫자 데이터를 k-평균 클러스터 멤버십으로 변환.

    이 변환기는 입력 데이터에 k-평균을 수행해 각 데이터 포인트를 가장 가까운 클러스터의 id로 변환한다.
    만약 목표 변수가 주어지면 유사한 데이터 포인트와 함께 grouping되고,
    분류 경계에 따르는 클러스터를 생성하기 위해 스케일링되고, k-평균 입력에 포함된다.
    """

    def __init__(self, k = 100, target_scale = 5.0, random_state = None):
        self.k = k
        self.target_scale = target_scale
        self.random_state = random_state

    def fit(self, X, y = None):
        """ 입력 데이터에 k-평균을 수행하고 중심점을 찾는다.
        """
        if y is None: # 목표 변수가 없으면 단순한 k-평균 수행
            km_model = KMeans(n_clusters = self.k, n_init = 20, random_state = self.random_state)
            km_model.fit(X)
            
            self.inertia_ = km_model.inertia_
            self.km_model = km_model
            self.cluster_centers_ = km_model.cluster_centers_
            return self

        # 목표 변수가 있으면, 적절한 스케일링을 적용하고, 이를 k-평균에 대한 입력 데이터에 포함시킨다.
        data_with_target = np.hstack((X, y[:, np.newaxis] * self.target_scale))
        # 데이터와 타겟에 대해 사전 학습할 k-평균 모델 구축
        km_model_pretrain = KMeans(n_clusters = self.k, n_init = 20, random_state = self.random_state)
        km_model_pretrain.fit(data_with_target)

        # k평균을 두번째로 실행해 목표 변수 없이 원시 공간에서 클러스터를 얻는다. 사전 학습을 통해 얻은 중심점을 활용해 초기화한다.
        # 반복을 통해 클러스터 할당과 중심점 계산을 다시 수행한다.

        km_model = KMeans(n_clusters = self.k, init = km_model_pretrain.cluster_centers_[:,:data_with_target.shape[1]-1], n_init = 1, max_iter = 1)

        km_model.fit(X)
        
        self.inertia_ = km_model.inertia_
        self.km_model = km_model
        self.cluster_centers_ = km_model.cluster_centers_
        return self

    def transform(self, X, y = None):
        """ 각 입력 데이터 포인트에 대해 가장 가까운 클러스터 ID 산출
        """
        clusters = self.km_model.predict(X)
        return clusters[:, np.newaxis]

    def fit_transform(self, X, y = None):
        self.fit(X, y)
        return self.transform(X, y)

In [175]:
kme = KMeansFeaturizer(random_state = 100)

In [176]:
features_k = kme.fit_transform(features__)

In [177]:
features_k_t = kme.transform(features_t__)

In [178]:
features_kmean = pd.concat([pd.DataFrame(features_k),pd.DataFrame(features_k_t)],axis = 0).reset_index().drop("index",axis= 1)

In [179]:
features_kmean = features_kmean.astype("str")

In [180]:
features_k_dum = pd.get_dummies(features_kmean)

In [181]:
max_col = features_k_dum.shape[1]
pca = PCA(n_components=max_col, random_state=0).fit(features_k_dum)

cumsum = np.cumsum(pca.explained_variance_ratio_)
num_col = np.argmax(cumsum >= 0.99) + 1


if num_col == 1:
    num_col = max_col

pca = PCA(n_components = num_col, random_state=0).fit_transform(features_k_dum)
features_k_dum = pd.DataFrame(pca)

In [182]:
features_k_train = features_k_dum[:len(features__)]

In [183]:
features_k_test = features_k_dum[len(features__):].reset_index().drop("index",axis = 1)

In [184]:
features__ = pd.concat([features__,features_k_train],axis = 1)

In [185]:
features_t__ = pd.concat([features_t__,features_k_test],axis = 1)

In [186]:
identifier = features__.columns.to_series().groupby(level=0).transform('cumcount')
features__.columns = identifier.astype('string')

identifier_t = features_t__.columns.to_series().groupby(level=0).transform('cumcount')
features_t__.columns = identifier_t.astype('string')

In [187]:
identifier = features__.columns.to_series().groupby(level=0).transform('cumcount')
features__.columns = features__.columns.astype('string') + identifier.astype('string')

identifier_t = features_t__.columns.to_series().groupby(level=0).transform('cumcount')
features_t__.columns = features_t__.columns.astype('string') + identifier_t.astype('string')

# 1등 피쳐 스케일링

In [120]:
#ji = pd.read_csv(os.path.abspath("../input")+"/지평오빠feature.csv")
#ji_t = pd.read_csv(os.path.abspath("../input")+"/지평오빠feature_te.csv")

In [121]:
#numeric_columns1 = ji.dtypes[ji.dtypes != 'object'].index.tolist()

In [122]:
"""
skewlist1 = []
numeric_columns_ns1 = []

for i in numeric_columns1:
    if 'SKEW' in i:
        skewlist1.append(i)
    else:
        numeric_columns_ns1.append(i)
"""

In [123]:
"""
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
ji[numeric_columns1] = scaler.fit_transform(ji[numeric_columns1])
ji_t[numeric_columns1] = scaler.transform(ji_t[numeric_columns1])
"""

# 마이피쳐 스케일링

In [188]:
numeric_columns2 = features__.dtypes[features__.dtypes != 'object'].index.tolist()

In [189]:
skewlist2 = []
numeric_columns_ns2 = []

for i in numeric_columns2:
    if 'SKEW' in i:
        skewlist2.append(i)
    else:
        numeric_columns_ns2.append(i)

In [190]:
from sklearn.preprocessing import StandardScaler

In [195]:
scaler = StandardScaler()
X = pd.concat([features__,features_t__])
X[numeric_columns2] = scaler.fit_transform(X[numeric_columns2])

In [126]:
"""
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
features__[numeric_columns2] = scaler.fit_transform(features__[numeric_columns2])
features_t__[numeric_columns2] = scaler.transform(features_t__[numeric_columns2])
"""

In [197]:
X[numeric_columns_ns2]

,00,01,02,03,04,05,06,07,08,09,...,0687,0688,0689,0690,0691,0692,0693,0694,0695,0696
0,-0.323552,-1.069081,0.323583,0.397445,0.316357,-0.278719,-0.436272,-0.440398,-0.441026,-0.427950,...,-0.026189,-0.039479,-2.022331e-08,-0.013704,-0.013424,-0.023435,-0.034805,-0.018872,-0.029333,-0.020232
1,1.095300,0.935383,-1.114245,-5.945645,-1.906480,-0.353342,-0.447218,-0.448054,-0.443736,-0.439160,...,18.978876,-8.002591,-4.722028e-06,-1.399423,-1.079595,-1.051805,-0.977831,-0.488823,-0.661692,-0.433071
2,0.209792,0.935383,-0.202216,-0.453076,-0.609746,-0.353342,0.151581,0.140264,0.155090,0.119534,...,-0.008668,-0.013122,-6.942517e-09,-0.004570,-0.004485,-0.007881,-0.011812,-0.006418,-0.010015,-0.006919
3,-0.323552,-1.069081,0.323583,0.397445,0.316357,-0.502589,-0.796968,-0.796618,-0.796793,-0.799234,...,-0.026189,-0.039479,-2.022330e-08,-0.013704,-0.013424,-0.023435,-0.034805,-0.018872,-0.029333,-0.020232
4,-0.323552,-1.069081,0.323583,0.397445,0.316357,-0.204095,0.204780,0.206993,0.221322,0.241479,...,-0.033832,-0.050907,-2.574278e-08,-0.017646,-0.017272,-0.030067,-0.044483,-0.024099,-0.037395,-0.025776
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14375,-0.323552,-1.069081,0.323583,0.397445,0.316357,-0.502589,-0.782550,-0.768118,-0.771346,-0.781234,...,-0.010971,-0.016599,-8.742721e-09,-0.005778,-0.005669,-0.009954,-0.014900,-0.008094,-0.012623,-0.008719
14376,-0.323552,-1.069081,0.323583,0.397445,0.316357,-0.502589,-0.889031,-0.884454,-0.883350,-0.881957,...,-0.018774,-0.028351,-1.471581e-08,-0.009855,-0.009661,-0.016913,-0.025214,-0.013683,-0.021302,-0.014703
14377,-0.323552,-1.069081,0.323583,0.397445,0.316357,-0.502589,-0.854250,-0.849032,-0.848337,-0.845273,...,-0.007558,-0.011445,-6.068004e-09,-0.003986,-0.003912,-0.006879,-0.010315,-0.005606,-0.008749,-0.006045
14378,-0.323552,-1.069081,0.323583,0.397445,0.316357,-0.427966,-0.854639,-0.842713,-0.845437,-0.851948,...,-0.008866,-0.013421,-7.097783e-09,-0.004674,-0.004586,-0.008060,-0.012078,-0.006562,-0.010239,-0.007074


In [198]:
X[skewlist2]

""
0
1
2
3
4
...
14375
14376
14377
14378


In [193]:
ALL = pd.DataFrame(X_all,columns=X_columns)
X_train_scaled = ALL[:21587]
X_test_scaled = ALL[21587:].reset_index().iloc[:,1:]

array([[-0.32355188, -1.0690808 ,  0.32358271, ..., -0.01887249,
        -0.02933255, -0.02023241],
       [ 1.09529969,  0.935383  , -1.11424512, ..., -0.48882288,
        -0.66169209, -0.4330707 ],
       [ 0.20979217,  0.935383  , -0.20221646, ..., -0.00641797,
        -0.01001452, -0.00691882],
       ...,
       [-0.32355188, -1.0690808 ,  0.32358271, ..., -0.00560557,
        -0.00874913, -0.00604523],
       [-0.32355188, -1.0690808 ,  0.32358271, ..., -0.00656232,
        -0.0102393 , -0.00707399],
       [-0.32355188, -1.0690808 ,  0.32358271, ..., -0.00656232,
        -0.0102393 , -0.00707399]])

# 마이피쳐 + 1st 피쳐 병합

In [127]:
data = pd.concat([ji[numeric_columns_ns1],ji[skewlist1], features__[numeric_columns_ns2], features__[skewlist2] ], axis=1)

features_t__.index = ji_t.index
data_te = pd.concat([ji_t[numeric_columns_ns1],ji_t[skewlist1] , features_t__[numeric_columns_ns2], features_t__[skewlist2] ], axis=1)

# 워드투벡 병합

In [199]:
w2v_list = [w2v_good,w2v_coner,w2v_brd]

In [200]:
data_all = X

In [205]:
data_all

,00,01,02,03,04,05,06,07,08,09,...,90,91,92,93,94,95,96,97,98,99
0,-0.323552,-1.069081,0.323583,0.397445,0.316357,-0.278719,-0.436272,-0.440398,-0.441026,-0.427950,...,-0.000623,0.104645,-0.047911,0.122851,0.175490,0.043112,0.096471,0.003035,0.016172,-0.120978
1,1.095300,0.935383,-1.114245,-5.945645,-1.906480,-0.353342,-0.447218,-0.448054,-0.443736,-0.439160,...,0.032947,0.116670,-0.029773,0.096462,0.194247,-0.029726,0.146337,-0.014407,0.121401,-0.060718
2,0.209792,0.935383,-0.202216,-0.453076,-0.609746,-0.353342,0.151581,0.140264,0.155090,0.119534,...,0.093265,0.124819,0.098818,0.207112,0.174643,-0.020386,0.037989,0.009853,0.013006,-0.041461
3,-0.323552,-1.069081,0.323583,0.397445,0.316357,-0.502589,-0.796968,-0.796618,-0.796793,-0.799234,...,0.026439,0.082867,-0.015591,0.069552,0.241776,0.263612,0.173288,-0.164216,-0.058703,-0.081887
4,-0.323552,-1.069081,0.323583,0.397445,0.316357,-0.204095,0.204780,0.206993,0.221322,0.241479,...,0.061685,0.078684,-0.005691,0.105126,0.174752,-0.032681,0.125656,-0.038001,0.049001,-0.112666
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35962,-0.323552,-1.069081,0.323583,0.397445,0.316357,-0.502589,-0.782550,-0.768118,-0.771346,-0.781234,...,0.010820,-0.136989,0.108609,0.016732,0.278335,0.001953,0.006035,0.036619,0.014452,-0.132841
35963,-0.323552,-1.069081,0.323583,0.397445,0.316357,-0.502589,-0.889031,-0.884454,-0.883350,-0.881957,...,0.028890,0.095177,-0.124906,0.093776,0.280154,0.028316,-0.026397,-0.010038,0.034472,-0.249929
35964,-0.323552,-1.069081,0.323583,0.397445,0.316357,-0.502589,-0.854250,-0.849032,-0.848337,-0.845273,...,0.008526,0.004818,-0.073347,0.165997,0.046893,-0.013805,0.002951,-0.080321,0.109302,-0.144372
35965,-0.323552,-1.069081,0.323583,0.397445,0.316357,-0.427966,-0.854639,-0.842713,-0.845437,-0.851948,...,0.120188,-0.151670,0.161709,0.157569,0.174731,-0.042550,0.070734,0.104473,0.027739,-0.084586


In [203]:
data_all = data_all.reset_index().drop("index",axis = 1)

In [204]:
for i in w2v_list:
    data_all = pd.concat([data_all,i],axis = 1)

In [208]:
data = data_all[:len(features__)]
data_te = data_all[len(features__):].reset_index().drop("index",axis = 1)

In [211]:
data.columns = data.columns.astype(str)
data_te.columns = data_te.columns.astype(str)

In [212]:
data2 = data.values
data_te2 = data_te.values

In [218]:
data.to_csv('4.min_1st_feature_0612.csv',index=False)

In [219]:
data_te.to_csv('4.min_1st_feature_te_0612.csv',index=False)

In [213]:
ftr = data2[:, 1:]
target = df_y_train
target_log = np.log1p(target)

TypeError: loop of ufunc does not support argument 0 of type str which has no callable log1p method

In [137]:
from sklearn.model_selection import train_test_split

train_x, valid_x, train_y, valid_y = train_test_split(ftr, target, test_size=0.2 , stratify = target , random_state=1000)
train_x.shape, valid_x.shape

((17269, 17125), (4318, 17125))

In [138]:
from lightgbm import LGBMRegressor

clf = LGBMRegressor(
        n_jobs=-1,
        n_estimators=1000,
        learning_rate=0.02,
        num_leaves=32,
        subsample=0.8,
        max_depth=12,
        silent=-1,
        verbose=-1
        )

clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], eval_metric = 'RMSE', 
        verbose=100, early_stopping_rounds= 50)

Training until validation scores don't improve for 50 rounds
[100]	training's rmse: 7.7676	training's l2: 60.3357	valid_1's rmse: 8.30134	valid_1's l2: 68.9122
[200]	training's rmse: 7.02968	training's l2: 49.4164	valid_1's rmse: 8.09159	valid_1's l2: 65.4738
[300]	training's rmse: 6.53169	training's l2: 42.663	valid_1's rmse: 8.05721	valid_1's l2: 64.9186
[400]	training's rmse: 6.12835	training's l2: 37.5567	valid_1's rmse: 8.05002	valid_1's l2: 64.8028
Early stopping, best iteration is:
[406]	training's rmse: 6.10618	training's l2: 37.2855	valid_1's rmse: 8.04892	valid_1's l2: 64.7852


LGBMRegressor(learning_rate=0.02, max_depth=12, n_estimators=1000,
              num_leaves=32, silent=-1, subsample=0.8, verbose=-1)

In [139]:
from sklearn.feature_selection import SelectFromModel
smf = SelectFromModel(clf, threshold='7.0*mean')
smf.fit(ftr, target)
X_new = smf.transform(ftr)
X_te_new = smf.transform(data_te2[:, 1:])
feature_idx = smf.get_support()
#feature_name = ftr.columns[feature_idx]

In [140]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(clf, X_new, target, scoring='neg_mean_squared_error', cv=5)
print('교차 검증별 정확도:', np.round(scores, 4))
print('평균 검증 정확도:', np.round(np.mean(scores), 4))
print('RMSE:', np.sqrt(-np.mean(scores)))
# mean - 7.0 : RMSE: 7.99449713498378
# mean - 5.0 : RMSE: RMSE: 8.000763598952103

교차 검증별 정확도: [-63.5007 -65.8621 -65.4368 -61.9738 -62.0358]
평균 검증 정확도: -63.7618
RMSE: 7.985101113600758


In [141]:
#pd.DataFrame(X_new).to_csv("features__2rd_jinew.csv")
#pd.DataFrame(X_te_new).to_csv("features_t__2rd_jinew.csv")